In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv('/content/AAPL.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
print(data.head())
print(data.info())
print(data['Open'].head())
data['Open'] = 1
print(data['Open'].head())
print(data['Open'].count())

scaler = MinMaxScaler(feature_range=(-1, 1)) 
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    #amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

series = scaler.fit_transform(data.to_numpy().reshape(-1, 1)).reshape(-1)
print(series)

                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2010-01-04  7.622500  7.660714  7.585000  7.643214   6.583586  493729600
2010-01-05  7.664286  7.699643  7.616071  7.656429   6.594968  601904800
2010-01-06  7.656429  7.686786  7.526786  7.534643   6.490066  552160000
2010-01-07  7.562500  7.571429  7.466071  7.520714   6.478067  477131200
2010-01-08  7.510714  7.571429  7.466429  7.570714   6.521136  447610800
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2010-01-04 to 2019-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2516 non-null   float64
 1   High       2516 non-null   float64
 2   Low        2516 non-null   float64
 3   Close      2516 non-null   float64
 4   Adj Close  2516 non-null   float64
 5   Volume     2516 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 13

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot

torch.manual_seed(0)
np.random.seed(0)

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)

input_window = 200 # number of input steps
output_window = 1 # number of prediction steps, in this model its fixed to one
batch_size = 10 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
          

class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

def get_data():
       
    from sklearn.preprocessing import MinMaxScaler
    
    #loading series data from a file
    from pandas import read_csv
    series = read_csv('/content/AAPL.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    open = series['Open'].to_numpy()
    high = series['High'].to_numpy()
    low = series['Low'].to_numpy()
    volume = series['Volume'].to_numpy()
    close = series['Close'].to_numpy()

    # looks like normalizing input values critical for the model
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    #amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

    series = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    close = scaler.fit_transform(close.reshape(-1, 1)).reshape(-1)
    high = scaler.fit_transform(high.reshape(-1, 1)).reshape(-1)
    low = scaler.fit_transform(low.reshape(-1, 1)).reshape(-1)
    volume = scaler.fit_transform(volume.reshape(-1, 1)).reshape(-1)
    #close = scaler.fit_transform(full_data['Close'].reshape(-1, 1)).reshape(-1)

    #split into train_data and test_data
    samples = 2516
    train_data = close[:int(samples*0.8)]
    test_data = close[int(samples*0.8):]


    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack.. 

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode \o/
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            output = eval_model(data)            
            total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy() -> no need to detach stuff.. 
    len(test_result)

    pyplot.plot(test_result,color="red", label = 'Test result')
    pyplot.plot(truth[:500],color="blue", label = 'Actual Close')
    pyplot.plot(test_result-truth,color="green", label = 'Test result - Actual Close')
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.legend(loc = 'lower left')
    pyplot.savefig('transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i


# predict the next n steps based on the input data 
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps):            
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    
    # I used this plot to visualize if the model picks up any long term structure within the data. 
    pyplot.plot(data,color="red", label = 'Predictions for next steps')       
    pyplot.plot(data[:input_window],color="blue", label = 'Close values')    
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.legend(loc = 'upper right')
    pyplot.savefig('transformer-future%d.png'%steps)
    pyplot.close()
        

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            total_loss += len(data[0])* criterion(output, targets).cpu().item()
    return total_loss / len(data_source)

train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

best_val_loss = float("inf")
epochs = 200 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data, 200)
    else:
        val_loss = evaluate(model, val_data)
   
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 

#src = torch.rand(input_window, batch_size, 1) # (source sequence length,batch size,feature number) 
#out = model(src)
#
#print(out)
#print(out.shape)


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |    36/  181 batches | lr 0.005000 | 24.76 ms | loss 7.78428 | ppl  2402.53
| epoch   1 |    72/  181 batches | lr 0.005000 | 20.54 ms | loss 0.03689 | ppl     1.04
| epoch   1 |   108/  181 batches | lr 0.005000 | 20.66 ms | loss 0.00628 | ppl     1.01
| epoch   1 |   144/  181 batches | lr 0.005000 | 20.49 ms | loss 0.00598 | ppl     1.01
| epoch   1 |   180/  181 batches | lr 0.005000 | 20.54 ms | loss 0.00762 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  4.01s | valid loss 0.30679 | valid ppl     1.36
-----------------------------------------------------------------------------------------
| epoch   2 |    36/  181 batches | lr 0.004513 | 21.79 ms | loss 0.08825 | ppl     1.09
| epoch   2 |    72/  181 batches | lr 0.004513 | 20.48 ms | loss 0.00761 | ppl     1.01
| epoch   2 |   108/  181 batches | lr 0.004513 | 20.59 ms | loss 0.00371 | ppl     1.00
| epoch   2 |   144/  181 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch  11 |    36/  181 batches | lr 0.002844 | 21.06 ms | loss 0.01613 | ppl     1.02
| epoch  11 |    72/  181 batches | lr 0.002844 | 20.53 ms | loss 0.00516 | ppl     1.01
| epoch  11 |   108/  181 batches | lr 0.002844 | 20.25 ms | loss 0.00181 | ppl     1.00
| epoch  11 |   144/  181 batches | lr 0.002844 | 20.39 ms | loss 0.00145 | ppl     1.00
| epoch  11 |   180/  181 batches | lr 0.002844 | 20.37 ms | loss 0.00143 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  11 | time:  3.85s | valid loss 0.03955 | valid ppl     1.04
-----------------------------------------------------------------------------------------
| epoch  12 |    36/  181 batches | lr 0.002702 | 21.43 ms | loss 0.01016 | ppl     1.01
| epoch  12 |    72/  181 batches | lr 0.002702 | 20.44 ms | loss 0.00371 | ppl     1.00
| epoch  12 |   108/  181 batches | lr 0.002702 | 20.47 ms | loss 0.00107 | ppl     1.00
| epoch  12 |   144/  181 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


...

| epoch 171 |    36/  181 batches | lr 0.000001 | 20.94 ms | loss 0.00020 | ppl     1.00
| epoch 171 |    72/  181 batches | lr 0.000001 | 20.50 ms | loss 0.00022 | ppl     1.00
| epoch 171 |   108/  181 batches | lr 0.000001 | 20.56 ms | loss 0.00017 | ppl     1.00
| epoch 171 |   144/  181 batches | lr 0.000001 | 20.45 ms | loss 0.00028 | ppl     1.00
| epoch 171 |   180/  181 batches | lr 0.000001 | 20.41 ms | loss 0.00032 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch 171 | time:  3.86s | valid loss 0.03399 | valid ppl     1.03
-----------------------------------------------------------------------------------------
| epoch 172 |    36/  181 batches | lr 0.000001 | 21.72 ms | loss 0.00020 | ppl     1.00
| epoch 172 |    72/  181 batches | lr 0.000001 | 20.35 ms | loss 0.00022 | ppl     1.00
| epoch 172 |   108/  181 batches | lr 0.000001 | 20.31 ms | loss 0.00017 | ppl     1.00
| epoch 172 |   144/  181 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch 181 |    36/  181 batches | lr 0.000000 | 20.85 ms | loss 0.00020 | ppl     1.00
| epoch 181 |    72/  181 batches | lr 0.000000 | 20.47 ms | loss 0.00022 | ppl     1.00
| epoch 181 |   108/  181 batches | lr 0.000000 | 20.24 ms | loss 0.00017 | ppl     1.00
| epoch 181 |   144/  181 batches | lr 0.000000 | 20.34 ms | loss 0.00028 | ppl     1.00
| epoch 181 |   180/  181 batches | lr 0.000000 | 20.59 ms | loss 0.00032 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch 181 | time:  3.85s | valid loss 0.03404 | valid ppl     1.03
-----------------------------------------------------------------------------------------
| epoch 182 |    36/  181 batches | lr 0.000000 | 21.73 ms | loss 0.00020 | ppl     1.00
| epoch 182 |    72/  181 batches | lr 0.000000 | 20.23 ms | loss 0.00022 | ppl     1.00
| epoch 182 |   108/  181 batches | lr 0.000000 | 20.28 ms | loss 0.00017 | ppl     1.00
| epoch 182 |   144/  181 batche

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch 191 |    36/  181 batches | lr 0.000000 | 20.76 ms | loss 0.00020 | ppl     1.00
| epoch 191 |    72/  181 batches | lr 0.000000 | 20.38 ms | loss 0.00022 | ppl     1.00
| epoch 191 |   108/  181 batches | lr 0.000000 | 20.35 ms | loss 0.00017 | ppl     1.00
| epoch 191 |   144/  181 batches | lr 0.000000 | 20.34 ms | loss 0.00028 | ppl     1.00
| epoch 191 |   180/  181 batches | lr 0.000000 | 20.36 ms | loss 0.00032 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch 191 | time:  3.84s | valid loss 0.03404 | valid ppl     1.03
-----------------------------------------------------------------------------------------
| epoch 192 |    36/  181 batches | lr 0.000000 | 21.44 ms | loss 0.00020 | ppl     1.00
| epoch 192 |    72/  181 batches | lr 0.000000 | 20.22 ms | loss 0.00022 | ppl     1.00
| epoch 192 |   108/  181 batches | lr 0.000000 | 20.23 ms | loss 0.00017 | ppl     1.00
| epoch 192 |   144/  181 batche